# * Ex-D Data

## Parameter

In [1]:
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import xlrd
import oracledb
import re
import FN_Monitoring as fn

config = configparser.ConfigParser()
config.read('../../my_config.ini')
config.sections()

pd.set_option('future.no_silent_downcasting', True)

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

AKPIPRD_user = config['AKPIPRD']['username']
AKPIPRD_pwd = config['AKPIPRD']['password']
AKPIPRD_db = config['AKPIPRD']['db']
AKPIPRD_host = config['AKPIPRD']['host']
AKPIPRD_port = config['AKPIPRD']['port']

curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')

## Actual : Monitoring
->  STG_KPI_NEWCO_DIAMOND_ACTUAL_INC

In [2]:
''' Execute Summany Data '''


# Input parameter
curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
# v_start_date = 20250201
print(f'\nParameter input...\n')
# print(f'   -> v_start_date: {v_start_date}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
print(f'\n{TDMDBPR_db} : Connected')
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/
        CASE WHEN REGEXP_LIKE(METRIC_CD, '^DB1R000900|^DB1R001000|^DB1S000101|^DB1S000109|^DB2R000500|^DB2S000100') 
            OR METRIC_CD IN (
                'DB0R00010001' --Total Inflow M1 : DTAC'
                , 'DB1S000100' --Prepay New sub
                , 'DB2S010100' --Postpaid activation sub B2C
                , 'DB2S020100' --Postpaid activation sub B2B
                --, 'DB1S000102' --Prepaid Activation Subs : Thai Mass : DTAC
                --, 'DB1S000103' --Prepaid Activation Subs : AEC/Migrants : DTAC
                --, 'DB1S000104' --Prepaid Activation Subs : Tourists (IN + OUT) : DTAC
                ) THEN 'Obsolete'
            ELSE 'Active'
            END KPI_FLAG
        , PAR_KEY, BASENAME, METRIC_CD, METRIC_NAME
        , CASE WHEN REGEXP_LIKE(METRIC_CD, 'A[A-K]$') THEN SUBSTR(METRIC_CD, -2) ELSE 'ALL' END CHANNEL_CD
        , MAX(LOAD_DATE) LOAD_DATE, MIN(TM_KEY_DAY) START_DAY, MAX(TM_KEY_DAY) END_DAY
        , SUM(CASE WHEN AREA_CD = 'P' THEN METRIC_VALUE END) AS P
        , SUM(CASE WHEN AREA_TYPE = 'G' THEN METRIC_VALUE END) AS G
        , SUM(CASE WHEN AREA_TYPE = 'H' THEN METRIC_VALUE END) AS H
        , SUM(CASE WHEN AREA_TYPE = 'HH' THEN METRIC_VALUE END) AS HH
        , SUM(CASE WHEN AREA_TYPE = 'CCAA' THEN METRIC_VALUE END) AS CCAA
        , SUM(CASE WHEN AREA_CD = '080' THEN METRIC_VALUE END) AS H_080
        , SUM(CASE WHEN AREA_CD = '016' THEN METRIC_VALUE END) AS H_016
        , SUM(CASE WHEN AREA_CD = '040' THEN METRIC_VALUE END) AS H_040
        , COUNT(DISTINCT AREA_TYPE) CNT_AREA_TYPE, COUNT(DISTINCT AREA_CD) CNT_AREA_CD, COUNT(1) ROW_CNT
    FROM GEOSPCAPPO.STG_KPI_NEWCO_DIAMOND_ACTUAL_INC
    GROUP BY PAR_KEY, BASENAME, METRIC_CD, METRIC_NAME
    --ORDER BY 1, PAR_KEY, BASENAME, METRIC_CD 
""")


try:
    execute_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
    print(f'\n   -> Execute query... {execute_datetime}')

    # Get : Summany DataFrame
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])
    print(f'\n   -> Summany DataFrame : {chk_src_df.shape[0]} rows, {chk_src_df.shape[1]} columns')

    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()
    print(f'\n{TDMDBPR_db} : Disconnected')


Parameter input...


TDMDBPR : Connected

   -> Execute query... 2025-03-26, 12:22:30

   -> Summany DataFrame : 132 rows, 20 columns

TDMDBPR : Disconnected


In [3]:
''' Revenue '''

# curr_mth = chk_src_df['TM_KEY_MTH'].max()
src_d_kpi_flag = 'Active'
rev_metric_list = [
    'DB1R000100' #Prepay Revenue
    , 'DB2R000100' #Postpaid Revenue
    , 'DB2R010100' #Postpaid Revenue B2C
    , 'DB2R020100' #Postpaid Revenue B2B
    ]

src_d_rev_overall_df = chk_src_df.loc[chk_src_df['KPI_FLAG']==src_d_kpi_flag].copy()
# src_d_rev_overall_df = src_d_rev_overall_df.loc[src_d_rev_overall_df['TM_KEY_MTH']==curr_mth]
src_d_rev_overall_df = src_d_rev_overall_df.loc[src_d_rev_overall_df['METRIC_CD'].isin(rev_metric_list)]
src_d_rev_overall_df = src_d_rev_overall_df.groupby(['PAR_KEY', 'METRIC_CD', 'METRIC_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'P':'sum', 'G':'sum', 'H':'sum', 'HH':'sum', 'CCAA':'sum', 'H_080':'sum', 'CNT_AREA_TYPE':'max', 'CNT_AREA_CD':'max', 'ROW_CNT':'sum'})
src_d_rev_overall_df = src_d_rev_overall_df.fillna(0).sort_values(by=['METRIC_CD']).reset_index()

mod_col_list = src_d_rev_overall_df.iloc[:, 6:12].columns.tolist()
for col in mod_col_list:
    src_d_rev_overall_df[col] = src_d_rev_overall_df[col].apply(lambda x: format(x, ',.0f'))

src_d_rev_overall_df

,PAR_KEY,METRIC_CD,METRIC_NAME,LOAD_DATE,START_DAY,END_DAY,P,G,H,HH,CCAA,H_080,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,20250323,DB1R000100,Prepay Revenue,2025-03-23,20240528,20250323,"5,870,190,649","10,621,169,006","10,613,408,630","10,613,408,630","5,870,190,357","98,713,171",5,1100,112072
1,20250323,DB2R000100,Postpaid Revenue,2025-03-23,20240528,20250323,"9,295,419,446","16,786,733,611","16,772,782,558","16,772,782,558","9,295,419,031","265,099,860",5,1100,112589
2,20250323,DB2R010100,Postpaid Revenue B2C,2025-03-23,20240528,20250323,"8,517,851,181","15,410,759,305","15,398,600,274","15,398,600,274","8,517,850,766","249,106,903",5,1100,112529
3,20250323,DB2R020100,Postpaid Revenue B2B,2025-03-23,20240528,20250323,"777,568,265","1,375,974,306","1,374,182,284","1,374,182,284","777,568,265","15,992,957",5,1100,73200


In [4]:
''' Subscription '''

src_d_kpi_flag = 'Active'
sub_metric_list = [
    'DB1S000500' #Prepay Usage Subs
    , 'DB1S000600' #Prepay Revenue Subs
    , 'DB2S000500' #Postpaid Active Subs
    , 'DB2S010500' #Postpaid Active Subs B2C
    , 'DB2S020400' #Postpaid Active Sub B2B
    ]

src_d_sub_overall_df = chk_src_df.loc[chk_src_df['KPI_FLAG']==src_d_kpi_flag].copy()
src_d_sub_overall_df = src_d_sub_overall_df.loc[src_d_sub_overall_df['METRIC_CD'].isin(sub_metric_list)]
src_d_sub_overall_df = src_d_sub_overall_df.groupby(['PAR_KEY', 'METRIC_CD', 'METRIC_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'P':'sum', 'G':'sum', 'H':'sum', 'HH':'sum', 'CCAA':'sum', 'H_080':'sum', 'CNT_AREA_TYPE':'max', 'CNT_AREA_CD':'max', 'ROW_CNT':'sum'})
src_d_sub_overall_df = src_d_sub_overall_df.fillna(0).sort_values(by=['METRIC_CD']).reset_index()

mod_col_list = src_d_sub_overall_df.iloc[:, 6:12].columns.tolist()
for col in mod_col_list:
    src_d_sub_overall_df[col] = src_d_sub_overall_df[col].apply(lambda x: format(x, ',.0f'))

src_d_sub_overall_df

,PAR_KEY,METRIC_CD,METRIC_NAME,LOAD_DATE,START_DAY,END_DAY,P,G,H,HH,CCAA,H_080,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,20250323,DB1S000500,Prepay Usage Subs,2025-03-23,20240528,20250323,"1,211,376,361","2,184,083,127","2,182,546,450","2,182,546,450","1,211,376,205","21,245,221",5,1100,113178
1,20250323,DB1S000600,Prepay Revenue Subs,2025-03-23,20240528,20250323,"640,449,011","1,158,930,454","1,158,025,740","1,158,025,740","640,448,954","11,335,618",5,1100,113178
2,20250323,DB2S000500,Postpaid Active Subs,2025-03-23,20240528,20250323,"594,257,292","1,073,519,589","1,072,603,425","1,072,603,425","594,257,149","16,064,172",5,1100,112994
3,20250323,DB2S010500,Postpaid Active Subs B2C,2025-03-23,20240528,20250323,"524,764,054","947,483,511","946,763,870","946,763,870","524,763,911","14,486,262",5,1100,112994
4,20250323,DB2S020400,Postpaid Active Sub B2B,2025-03-23,20240528,20250323,"69,493,238","126,036,078","125,839,555","125,839,555","69,493,238","1,577,910",5,1100,112358


## Actual : Transaction
->  STG_KPI_NEWCO_DIAMOND_ACTUAL_INC

In [5]:
''' Execute Daily by case '''


# Input parameter
curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
v_start_date = 20240101
print(f'\nParameter input...\n')
print(f'   -> v_start_date: {v_start_date}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
print(f'\n{TDMDBPR_db} : Connected')
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/
        PAR_KEY, METRIC_CD, METRIC_NAME, TM_KEY_DAY
        , MAX(LOAD_DATE) LOAD_DATE
        , SUM(CASE WHEN AREA_CD = 'P' THEN METRIC_VALUE END) AS P
        , SUM(CASE WHEN AREA_TYPE = 'G' THEN METRIC_VALUE END) AS G
        , SUM(CASE WHEN AREA_TYPE = 'H' THEN METRIC_VALUE END) AS H
        , SUM(CASE WHEN AREA_TYPE = 'HH' THEN METRIC_VALUE END) AS HH
        , SUM(CASE WHEN AREA_TYPE = 'CCAA' THEN METRIC_VALUE END) AS CCAA
        , SUM(CASE WHEN AREA_CD = '080' THEN METRIC_VALUE END) AS H_080
        , SUM(CASE WHEN AREA_CD = '016' THEN METRIC_VALUE END) AS H_016
        , SUM(CASE WHEN AREA_CD = '040' THEN METRIC_VALUE END) AS H_040
        , COUNT(DISTINCT AREA_TYPE) CNT_AREA_TYPE, COUNT(DISTINCT AREA_CD) CNT_AREA_CD, COUNT(1) ROW_CNT
    FROM GEOSPCAPPO.STG_KPI_NEWCO_DIAMOND_ACTUAL_INC
    WHERE METRIC_CD IN (
        'DB1R000100' --Prepay Revenue
        , 'DB2R000100' --Postpaid Revenue
        , 'DB2R010100' --Postpaid Revenue B2C
        , 'DB2R020100' --Postpaid Revenue B2B
        , 'DB1S000500' --Prepay Usage Subs
        , 'DB1S000600' --Prepay Revenue Subs
        , 'DB2S000500' --Postpaid Active Subs
        , 'DB2S010500' --Postpaid Active Subs B2C
        , 'DB2S020400' --Postpaid Active Sub B2B
        )
    AND TM_KEY_DAY >= {v_start_date}
    GROUP BY PAR_KEY, METRIC_CD, METRIC_NAME, TM_KEY_DAY
""")


try:
    execute_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
    print(f'\n   -> Execute query... {execute_datetime}')

    # Get : Summary DataFrame
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])
    print(f'\n   -> Daily DataFrame : {chk_src_df.shape[0]} rows, {chk_src_df.shape[1]} columns')

    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()
    print(f'\n{TDMDBPR_db} : Disconnected')


Parameter input...

   -> v_start_date: 20240101

TDMDBPR : Connected

   -> Execute query... 2025-03-26, 12:23:48

   -> Daily DataFrame : 908 rows, 16 columns

TDMDBPR : Disconnected


In [6]:
''' DB1R000100 : Prepay Revenue '''

v_start_day = 20240101
v_metric_cd = 'DB1R000100'
print(f'\nDB1R000100 : Prepay Revenue\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
print(f'   -> v_start_day: {v_start_day}')

DB1R000100_day__df = chk_src_df.copy()
DB1R000100_day__df = DB1R000100_day__df.loc[DB1R000100_day__df['TM_KEY_DAY']>=v_start_day]
DB1R000100_day__df = DB1R000100_day__df.loc[DB1R000100_day__df['METRIC_CD']==v_metric_cd]
DB1R000100_day__df = DB1R000100_day__df.fillna(0).sort_values(by=['METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = DB1R000100_day__df.iloc[:, 5:13].columns.tolist()
for col in mod_col_list:
    DB1R000100_day__df[col] = DB1R000100_day__df[col].apply(lambda x: format(x, ',.0f'))

DB1R000100_day__df


DB1R000100 : Prepay Revenue

   -> v_metric_cd: DB1R000100
   -> v_start_day: 20240101


,PAR_KEY,METRIC_CD,METRIC_NAME,TM_KEY_DAY,LOAD_DATE,P,G,H,HH,CCAA,H_080,H_016,H_040,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,20250323,DB1R000100,Prepay Revenue,20240528,2024-06-06,"57,081,475","57,081,475","57,081,459","57,081,459","57,081,459",0,"303,238","890,374",5,1099,1111
1,20250323,DB1R000100,Prepay Revenue,20240529,2024-06-07,"54,750,640","54,750,640","54,750,601","54,750,601","54,750,601",0,"320,514","861,052",5,1099,1111
2,20250323,DB1R000100,Prepay Revenue,20240530,2024-06-08,"59,115,506","59,115,506","59,115,490","59,115,490","59,115,490",0,"312,960","934,515",5,1099,1111
3,20250323,DB1R000100,Prepay Revenue,20240531,2024-06-09,"70,858,400","70,858,400","70,858,378","70,858,378","70,858,378",0,"382,069","1,193,650",5,1099,1111
4,20250323,DB1R000100,Prepay Revenue,20240601,2024-06-10,"70,350,955","70,350,955","70,350,942","70,350,942","70,350,942",0,"385,655","1,166,659",5,1099,1111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,20250323,DB1R000100,Prepay Revenue,20250318,2025-03-23,"52,897,820","105,795,640","105,710,180","105,710,180","52,897,820","1,161,342",0,0,5,1098,1123
96,20250323,DB1R000100,Prepay Revenue,20250319,2025-03-23,"51,900,182","103,800,365","103,713,430","103,713,430","51,900,182","1,145,778",0,0,5,1098,1123
97,20250323,DB1R000100,Prepay Revenue,20250320,2025-03-23,"54,132,906","108,265,812","108,186,179","108,186,179","54,132,906","1,191,057",0,0,5,1098,1123
98,20250323,DB1R000100,Prepay Revenue,20250322,2025-03-23,"53,245,173","106,490,345","106,403,366","106,403,366","53,245,173","1,102,018",0,0,5,1098,1123


In [7]:
''' DB2R000100 : Postpaid Revenue '''

v_start_day = 20240101
v_metric_cd = 'DB2R000100'
print(f'\nDB2R000100 : Postpaid Revenue\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
print(f'   -> v_start_day: {v_start_day}')

DB2R000100_day__df = chk_src_df.copy()
DB2R000100_day__df = DB2R000100_day__df.loc[DB2R000100_day__df['TM_KEY_DAY']>=v_start_day]
DB2R000100_day__df = DB2R000100_day__df.loc[DB2R000100_day__df['METRIC_CD']==v_metric_cd]
DB2R000100_day__df = DB2R000100_day__df.fillna(0).sort_values(by=['METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = DB2R000100_day__df.iloc[:, 5:13].columns.tolist()
for col in mod_col_list:
    DB2R000100_day__df[col] = DB2R000100_day__df[col].apply(lambda x: format(x, ',.0f'))

DB2R000100_day__df


DB2R000100 : Postpaid Revenue

   -> v_metric_cd: DB2R000100
   -> v_start_day: 20240101


,PAR_KEY,METRIC_CD,METRIC_NAME,TM_KEY_DAY,LOAD_DATE,P,G,H,HH,CCAA,H_080,H_016,H_040,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,20250323,DB2R000100,Postpaid Revenue,20240528,2024-06-06,"93,542,212","93,542,212","93,542,212","93,542,212","93,542,212",0,"1,056,671","2,289,788",5,1099,1108
1,20250323,DB2R000100,Postpaid Revenue,20240529,2024-06-07,"281,666,951","281,666,951","281,666,951","281,666,951","281,666,951",0,"3,264,840","6,703,783",5,1099,1108
2,20250323,DB2R000100,Postpaid Revenue,20240530,2024-06-08,"6,554,985","6,554,985","6,554,985","6,554,985","6,554,985",0,"72,122","128,142",5,1099,1102
3,20250323,DB2R000100,Postpaid Revenue,20240531,2024-06-09,"6,595,697","6,595,697","6,595,697","6,595,697","6,595,697",0,"87,719","144,124",5,1099,1103
4,20250323,DB2R000100,Postpaid Revenue,20240601,2024-06-10,"8,733,426","8,733,426","8,733,426","8,733,426","8,733,426",0,"90,050","188,734",5,1099,1103
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,20250323,DB2R000100,Postpaid Revenue,20250319,2025-03-23,"84,627,845","169,255,690","169,128,239","169,128,239","84,627,845","2,901,099",0,0,5,1098,1115
97,20250323,DB2R000100,Postpaid Revenue,20250320,2025-03-23,"80,844,670","161,689,341","161,564,238","161,564,238","80,844,670","2,913,927",0,0,5,1097,1116
98,20250323,DB2R000100,Postpaid Revenue,20250321,2025-03-23,"84,041,042","168,082,084","167,947,153","167,947,153","84,041,042","2,893,200",0,0,5,1097,1117
99,20250323,DB2R000100,Postpaid Revenue,20250322,2025-03-23,"81,527,616","163,055,232","162,920,519","162,920,519","81,527,616","2,886,803",0,0,5,1098,1120


In [8]:
''' DB2R010100 : Postpaid Revenue B2C '''

v_start_day = 20240101
v_metric_cd = 'DB2R010100'
print(f'\nDB2R010100 : Postpaid Revenue B2C\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
print(f'   -> v_start_day: {v_start_day}')

DB2R010100_day__df = chk_src_df.copy()
DB2R010100_day__df = DB2R010100_day__df.loc[DB2R010100_day__df['TM_KEY_DAY']>=v_start_day]
DB2R010100_day__df = DB2R010100_day__df.loc[DB2R010100_day__df['METRIC_CD']==v_metric_cd]
DB2R010100_day__df = DB2R010100_day__df.fillna(0).sort_values(by=['METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = DB2R010100_day__df.iloc[:, 5:13].columns.tolist()
for col in mod_col_list:
    DB2R010100_day__df[col] = DB2R010100_day__df[col].apply(lambda x: format(x, ',.0f'))

DB2R010100_day__df


DB2R010100 : Postpaid Revenue B2C

   -> v_metric_cd: DB2R010100
   -> v_start_day: 20240101


,PAR_KEY,METRIC_CD,METRIC_NAME,TM_KEY_DAY,LOAD_DATE,P,G,H,HH,CCAA,H_080,H_016,H_040,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,20250323,DB2R010100,Postpaid Revenue B2C,20240528,2024-06-06,"90,417,597","90,417,597","90,417,597","90,417,597","90,417,597",0,"1,054,690","2,284,168",5,1098,1107
1,20250323,DB2R010100,Postpaid Revenue B2C,20240529,2024-06-07,"280,520,688","280,520,688","280,520,688","280,520,688","280,520,688",0,"3,262,211","6,695,846",5,1099,1108
2,20250323,DB2R010100,Postpaid Revenue B2C,20240530,2024-06-08,"4,883,710","4,883,710","4,883,710","4,883,710","4,883,710",0,"70,826","122,243",5,1099,1101
3,20250323,DB2R010100,Postpaid Revenue B2C,20240531,2024-06-09,"5,308,594","5,308,594","5,308,594","5,308,594","5,308,594",0,"86,421","134,865",5,1099,1101
4,20250323,DB2R010100,Postpaid Revenue B2C,20240601,2024-06-10,"5,843,787","5,843,787","5,843,787","5,843,787","5,843,787",0,"84,771","173,885",5,1099,1102
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,20250323,DB2R010100,Postpaid Revenue B2C,20250319,2025-03-23,"81,477,760","162,955,520","162,828,651","162,828,651","81,477,760","2,887,896",0,0,5,1098,1115
97,20250323,DB2R010100,Postpaid Revenue B2C,20250320,2025-03-23,"79,624,587","159,249,173","159,125,261","159,125,261","79,624,587","2,906,121",0,0,5,1097,1116
98,20250323,DB2R010100,Postpaid Revenue B2C,20250321,2025-03-23,"80,669,451","161,338,903","161,204,709","161,204,709","80,669,451","2,885,014",0,0,5,1097,1116
99,20250323,DB2R010100,Postpaid Revenue B2C,20250322,2025-03-23,"80,626,045","161,252,090","161,117,646","161,117,646","80,626,045","2,879,754",0,0,5,1098,1119


In [9]:
''' DB2R020100 : Postpaid Revenue B2B '''

v_start_day = 20240101
v_metric_cd = 'DB2R020100'
print(f'\nDB2R020100 : Postpaid Revenue B2B\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
print(f'   -> v_start_day: {v_start_day}')

DB2R020100_day__df = chk_src_df.copy()
DB2R020100_day__df = DB2R020100_day__df.loc[DB2R020100_day__df['TM_KEY_DAY']>=v_start_day]
DB2R020100_day__df = DB2R020100_day__df.loc[DB2R020100_day__df['METRIC_CD']==v_metric_cd]
DB2R020100_day__df = DB2R020100_day__df.fillna(0).sort_values(by=['METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = DB2R020100_day__df.iloc[:, 5:13].columns.tolist()
for col in mod_col_list:
    DB2R020100_day__df[col] = DB2R020100_day__df[col].apply(lambda x: format(x, ',.0f'))

DB2R020100_day__df


DB2R020100 : Postpaid Revenue B2B

   -> v_metric_cd: DB2R020100
   -> v_start_day: 20240101


,PAR_KEY,METRIC_CD,METRIC_NAME,TM_KEY_DAY,LOAD_DATE,P,G,H,HH,CCAA,H_080,H_016,H_040,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,20250323,DB2R020100,Postpaid Revenue B2B,20240528,2024-06-06,"3,124,614","3,124,614","3,124,614","3,124,614","3,124,614",0,"1,981","5,620",5,726,729
1,20250323,DB2R020100,Postpaid Revenue B2B,20240529,2024-06-07,"1,146,263","1,146,263","1,146,263","1,146,263","1,146,263",0,"2,629","7,936",5,734,737
2,20250323,DB2R020100,Postpaid Revenue B2B,20240530,2024-06-08,"1,671,275","1,671,275","1,671,275","1,671,275","1,671,275",0,"1,296","5,899",5,740,743
3,20250323,DB2R020100,Postpaid Revenue B2B,20240531,2024-06-09,"1,287,104","1,287,104","1,287,104","1,287,104","1,287,104",0,"1,298","9,258",5,780,783
4,20250323,DB2R020100,Postpaid Revenue B2B,20240601,2024-06-10,"2,889,639","2,889,639","2,889,639","2,889,639","2,889,639",0,"5,279","14,848",5,731,734
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,20250323,DB2R020100,Postpaid Revenue B2B,20250319,2025-03-23,"3,150,085","6,300,170","6,299,588","6,299,588","3,150,085","13,203",0,0,5,807,822
97,20250323,DB2R020100,Postpaid Revenue B2B,20250320,2025-03-23,"1,220,084","2,440,168","2,438,977","2,438,977","1,220,084","7,806",0,0,5,798,813
98,20250323,DB2R020100,Postpaid Revenue B2B,20250321,2025-03-23,"3,371,590","6,743,181","6,742,444","6,742,444","3,371,590","8,186",0,0,5,816,831
99,20250323,DB2R020100,Postpaid Revenue B2B,20250322,2025-03-23,"901,571","1,803,142","1,802,873","1,802,873","901,571","7,050",0,0,5,766,781


In [10]:
# txn_metric_list = [
#     'DB1R000100' #Prepay Revenue
#     , 'DB2R000100' #Postpaid Revenue
#     , 'DB2R010100' #Postpaid Revenue B2C
#     , 'DB2R020100' #Postpaid Revenue B2B
#     , 'DB1S000500' #Prepay Usage Subs
#     , 'DB1S000600' #Prepay Revenue Subs
#     , 'DB2S000500' #Postpaid Active Subs
#     , 'DB2S010500' #Postpaid Active Subs B2C
#     , 'DB2S020400' #Postpaid Active Sub B2B
#     ]